# 0.准备工作

大模型训练配置要求较高，我们在AutoDL网站上租用了服务器，中途曾多次更换不同配置的服务器，综合考虑模型训练时间和金钱的成本，最终主要使用的服务器配置如下：

![pic0](pictures/pic0.png)

上传代码文件到服务器上，进入`src`目录，根据`requirements.txt`安装必备库。

In [ ]:
%cd src
pip install -r requirements.txt --upgrade

大模型下载时文件较大，如果要更改下载的大模型保存位置，方法如下：

In [ ]:
!vim ~/.bashrc

将以下内容保存到`~/.bashrc`文件中

`export HUGGINGFACE_HUB_CACHE=保存位置
export TRANSFORMERS_CACHE=保存位置`

“保存位置”要自己修改。

In [ ]:
!source ~/.bashrc

后面训练的时候也要加上`cache_dir=保存位置`的参数。

准备工作阶段完成。

## 0.5.快速测试

因为base model文件过大，所以实验文件不包含base model。本notebook记录了详细的工作复现流程，但如果您非常熟悉这些工作，并且只是想快速测试模型效果，可直接下载模型[shibing624/ziya-llama-13b-medical-merged](https://huggingface.co/shibing624/ziya-llama-13b-medical-merged)，然后到**2.增量预训练**和**3.有监督微调**这两章，找到合并模型的部分合并lora模型（**请注意检查读取保存模型和数据的目录！**），合并完就可直接到**4.模型测试**一章进行测试。

# 1.数据预处理

查看数据集。

![pic1](pictures/pic1.png)

由于文件名显示乱码，故将文件名根据数据条数分别改为`40474.xlsx`和`2500.tsv`，并将关键内容字符串（标准化术语）都分别转换存储到`40474.txt`和`2500.txt`文本文件里，位于`src/data/pretrain`，作为**预训练(PreTraining, PT)**数据，为模型注入标准化术语的知识。`2500.txt`可直接由`2500.tsv`修改拓展名得到，`40474.txt`由以下代码从`40474.xlsx`转换得到：

In [ ]:
import openpyxl

# 打开Excel文件
workbook = openpyxl.load_workbook('40474.xlsx')

# 选择工作表
sheet = workbook.active

# 打开一个文本文件来写入内容
with open('40474.txt', 'w', encoding='UTF-8') as txt_file:
    # 遍历第二列内容并写入文本文件
    for row in sheet.iter_rows(min_col=2, max_col=2, values_only=True):
        for cell_value in row:
            if cell_value is not None:
                txt_file.write(str(cell_value) + '\n')

用于**有监督微调(Supervised FineTuning, SFT)**的数据包括`train.json`和`dev.json`，我们使用了以下代码转换为大模型训练所需要的jsonl格式：

In [ ]:
import json

def json2jsonl(input_name, output_name):
    # 读取输入 JSON 文件
    with open(input_name, 'r', encoding='UTF-8') as input_file:
        data = json.load(input_file)

    # 转换数据格式
    output_data = []
    for item in data:
        item = [
            {"from": "human", "value": item["text"]},
            {"from": "gpt", "value": item["normalized_result"]}
        ]
        output_data.append({"conversations": item})

    # 将转换后的数据写入输出 JSON 文件
    with open(output_name, 'w', encoding='UTF-8') as output_file:
        for item in output_data:
            json.dump(item, output_file, ensure_ascii=False)
            output_file.write("\n")
    
json2jsonl('train.json', 'train.jsonl')
json2jsonl('dev.json', 'dev.jsonl')

由于大模型预测较慢，几秒才能推理出一个结果，如果使用全部验证集来验证模型预测准确率，时间过长，故仅选用2000条数据验证集的最后100个数据来进行验证，其余1900条数据并入训练集。这个处理比较简单，是人工直接分割`dev.jsonl`为`dev100.jsonl`和`dev1900.jsonl`。`train.jsonl`和`dev1900.jsonl`放入`src/data/pretrain`作为训练集，`dev100.jsonl`将每一个问题按行存储到`dev100.txt`，`dev100.txt`放入`data/test`文件夹作为验证集，`dev100.jsonl`保留用于计算**Micro-F1**值。以下是将`dev100.jsonl`转换成`dev100.txt`的代码：

In [ ]:
import json

# 打开输出文件以写入数据
with open('dev100.txt', 'w', encoding='UTF-8') as output_file:
    # 逐行读取JSONL文件
    with open('dev100.jsonl', 'r', encoding='UTF-8') as input_file:
        for line in input_file:
            data = json.loads(line)
            conversations = data.get("conversations", [])

            # 遍历对话
            for conv in conversations:
                if conv.get("from") == "human":
                    value = conv.get("value")
                    # 将来自"human"的值写入文本文件，一句一行
                    output_file.write(value + '\n')

数据预处理阶段完成。

# 2. 增量预训练

增量预训练，在海量领域文本数据上二次预训练GPT模型，以注入领域知识。

底座生成模型使用的是类型为llama的[shibing624/ziya-llama-13b-medical-merged](https://huggingface.co/shibing624/ziya-llama-13b-medical-merged)模型。shibing624/ziya-llama-13b-medical-merged模型在中文开放测试集中的表现优异，继承了两方面的优势：1）微调训练的底座是[Ziya-LLaMA-13B](https://huggingface.co/IDEA-CCNL/Ziya-LLaMA-13B-v1)模型，是较强的中英文底座模型，2）微调使用的是高质量240万条中英文医疗指令数据集，和多种通用指令数据集，微调后的模型在医疗行业答复能力达到领先水平，在通用问题上的答复能力不弱于LLaMA-13B。

数据集PT阶段使用的是`40474.txt`和`2500.txt`，位于`src/data/pretrain`文件夹。

下面的代码是进行预训练的命令，执行前请仔细检查参数，尤其请检查`--cache_dir`参数，是大模型下载保存位置，如果指定位置无相应大模型则会自动从HuggingFace下载（需要使用学术网络加速方法才能访问和下载，如果使用AutoDL服务器可参考帮助文档），这个保存位置应与开头在`~/.bashrc`设置的保存位置相一致。这个模型大约25GB，后面还会合并两次模型，请注意选择一个容量大的区域，建议在该区域至少预留80GB。我们在开头介绍的服务器配置下执行以下代码训练了约3.5小时。

确认训练集和模型位置等参数无误后，在`src`文件夹下执行代码：

In [ ]:
!python pretraining.py \
    --model_type llama \
    --model_name_or_path shibing624/ziya-llama-13b-medical-merged \
    --cache_dir /root/autodl-tmp \
    --train_file_dir ./data/pretrain \
    --validation_file_dir ./data/pretrain \
    --per_device_train_batch_size 16 \
    --per_device_eval_batch_size 16 \
    --do_train \
    --do_eval \
    --use_peft True \
    --seed 42 \
    --fp16 \
    --max_train_samples -1 \
    --max_eval_samples -1 \
    --num_train_epochs 20 \
    --learning_rate 2e-4 \
    --warmup_ratio 0.05 \
    --weight_decay 0.01 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --evaluation_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 64 \
    --block_size 1024 \
    --output_dir outputs-pt-v2 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

模型训练结果：

使用lora训练模型，则保存的lora权重是`adapter_model.bin`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`。日志保存在`--output_dir`目录的`runs`子目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`。

对日志记录的eval loss值随epoch变化情况进行可视化如下（y轴是对数刻度）：

![pic2](pictures/pic2.png)

lora模型权重合并到base model，合并后的模型保存在`--output_dir`参数对应的目录下，合并后的模型大约25GB，请注意检查该参数以防磁盘分区容量不足，合并方法如下：

In [ ]:
!python merge_peft_adapter.py \
    --model_type llama \
    --base_model_name_or_path shibing624/ziya-llama-13b-medical-merged \ 
    --peft_model_path outputs-pt-v2 \
    --output_dir /root/autodl-tmp/merged-pt-v2/

PT阶段增量预训练完成。

# 3.有监督微调

有监督微调，构造指令微调数据集，在预训练模型基础上做指令精调，以对齐指令意图。

底座生成模型使用的是增量预训练阶段得到的预训练模型，即为`merge_peft_adapter.py`合并生成的模型，保存位置在它`--output_dir`参数对应的目录下。

数据集SFT阶段使用的是`train.jsonl`和`dev100.jsonl`，位于`src/data/finetune`文件夹。

与PT阶段类似，确认训练集和模型位置等参数无误后，在`src`文件夹下执行代码（我们在开头介绍的服务器配置下执行以下代码训练了约21小时）：

In [ ]:
!python supervised_finetuning.py \
    --model_type llama \
    --model_name_or_path /root/autodl-tmp/merged-pt-v2 \
    --train_file_dir ./data/finetune \
    --validation_file_dir ./data/finetune \
    --per_device_train_batch_size 64 \
    --per_device_eval_batch_size 64 \
    --do_train \
    --do_eval \
    --use_peft True \
    --fp16 \
    --max_train_samples -1 \
    --max_eval_samples -1 \
    --num_train_epochs 50 \
    --learning_rate 5e-5 \
    --warmup_ratio 0.05 \
    --weight_decay 0.05 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --evaluation_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 64 \
    --output_dir outputs-sft-v2 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True \
    --cache_dir /root/autodl-tmp

模型训练结果：

使用lora训练模型，则保存的lora权重是`adapter_model.bin`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`。日志保存在`--output_dir`目录的`runs`子目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`。

对日志记录的eval loss值随epoch变化情况进行可视化如下（y轴是对数刻度）：

![pic3](pictures/pic3.png)

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [ ]:
!python merge_peft_adapter.py 
    --model_type llama \
    --base_model_name_or_path /root/autodl-tmp/merged-pt-v2 \
    --peft_model_path outputs-sft-v2 \
    --output_dir /root/autodl-tmp/merged-sft-v2/

SFT阶段有监督微调完成。

# 4.模型测试

接下来就可以测试模型了。请注意调整参数。`--base_model`是模型的保存位置，一般来说需要修改成自己的模型位置；`--data_file`是测试数据保存位置；`--predictions_file`是预测结果保存位置；其它参数不用修改。

In [ ]:
!python inference.py
    --model_type llama \
    --base_model /root/autodl-tmp/merged-sft-v2 \
    --data_file ./data/test/dev100.txt \
    --predictions_file /root/autodl-tmp/dev100_predict_v2.jsonl \
    --max_new_tokens 128

预测得到了`dev100_predict.jsonl`文件，我们需要用这个文件和`dev100.jsonl`计算Micro-F1值，以衡量模型效果。

Micro-F1计算公式如下：

$$Precision=\frac{\sum^{n}_{i=1}TP_i}{\sum^{n}_{i=1}TP_i+\sum^{n}_{i=1}FP_i}$$

$$Recall=\frac{\sum^{n}_{i=1}TP_i}{\sum^{n}_{i=1}TP_i+\sum^{n}_{i=1}FN_i}$$

$$F1_{micro}=2\cdot\frac{Precision\cdot Recall}{Precision+Recall}$$

其中$TP_i$是第$i$条数据预测正确的词数，$FP_i$是第$i$条数据预测了但真实数据没有的词数，$FN_i$是第$i$条数据真实数据有但没预测出来的词数。

因为Micro-F1考虑了各种类别的数量，所以更适用于数据分布不平衡的情况。在这种情况下，数量较多的类别对F1的影响会较大。

以下是根据真实数据`dev100.jsonl`和预测数据`dev100_predict_v2.jsonl`计算Micro-F1的代码：

In [ ]:
import json

# 打开输入文件和输出文件
with open('new_dev100.jsonl', 'r', encoding='utf-8') as true_file, \
        open('dev100_predict_v2.jsonl', 'r', encoding='utf-8') as pred_file, \
        open('dev100_eval_v2.jsonl', 'w', encoding='utf-8') as output_file:
    # 初始化数值用于计算Micro-F1
    total_TP, total_FP, total_FN = 0, 0, 0

    # 逐行读取输入文件
    for true_line, pred_line in zip(true_file, pred_file):
        true_data = json.loads(true_line)
        pred_data = json.loads(pred_line)

        # 获取来自gpt的value值
        true_value = true_data['conversations'][1]['value']
        pred_value = pred_data['Output']

        # 根据井号分割字符串为列表（用set去重）
        true_values_list = list(set(true_value.split('##')))
        pred_values_list = list(set(pred_value.split('##')))

        # 计算TP, FP, FN
        TP, FP, FN = 0, 0, 0
        for true_item in true_values_list:
            for pred_item in pred_values_list:
                if true_item == pred_item:
                    TP += 1
        FP = len(pred_values_list) - TP
        FN = len(true_values_list) - TP
        total_TP += TP
        total_FP += FP
        total_FN += FN

        # 创建新的数据格式
        new_data = {"TP": TP, "FP": FP, "FN": FN,
                    "true": true_values_list, "pred": pred_values_list}

        # 将新数据写入输出文件
        output_file.write(json.dumps(new_data, ensure_ascii=False) + '\n')

# 计算Micro-F1
precision = total_TP / (total_TP + total_FP)
recall = total_TP / (total_TP + total_FN)
micro_F1 = 2 * precision * recall / (precision + recall)

print(f"precision: {precision}, recall: {recall}, micro-F1: {micro_F1}")

经以上代码计算得到`precision: 0.680628272251309, recall: 0.6842105263157895, micro-F1: 0.6824146981627297`，也就是说Micro-F1约0.7。

每条数据分割后的标准化词列表，以及$TP$,$FP$,$FN$都可以在文件`dev100_eval_v2.jsonl`中查看。

![pic4](pictures/pic4.png)

模型测试阶段完成。

# 5.后记

第一次测试的时候PT训练了1个epochs，SFT阶段训练了10个epochs，训练集和验证集比例一样，可用`outputs-pt-v2`和`outputs-sft-v2`按顺序与基座模型`shibing624/ziya-llama-13b-medical-merged`合并得到。它得到的预测评价结果是`precision: 0.6836734693877551, recall: 0.7052631578947368, micro-F1: 0.694300518134715`。

本作业参考代码仓库[https://github.com/shibing624/MedicalGPT/tree/main](https://github.com/shibing624/MedicalGPT/tree/main)。